# Multi-Layer Perceptron Attack
TJ Kim, 12 Nov 2019

We build a multi-layer perceptron model to classify adult.csv data. Then we perform an equation solving attack. Then, a uniform query attack will be performed, as well as an adaptive retraining attack.

## Importing Data
First we import CSV data as pandas and divide to test and training set.

In [8]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Import Dataset
filename = "adult_encoded.csv"
df = pd.read_csv(filename, sep='\s*,\s*',engine = 'python')
df = df.loc[:,df.columns != 'Unnamed: 0']

# Separate each dataset into training, testing, and query data
total_rest,total_test = train_test_split(df, random_state=30, test_size=0.2, shuffle=True)
total_train,queries = train_test_split(total_rest,random_state = 30, test_size = 0.4, shuffle = True)

# Separate each sub dataset to input and output
total_train_data = total_train.loc[:,total_train.columns != 'income_over_50k']
total_train_label = total_train.loc[:,total_train.columns =='income_over_50k']
total_test_data = total_train.loc[:,total_test.columns != 'income_over_50k']
total_test_label = total_train.loc[:,total_test.columns =='income_over_50k']
query_data = queries.loc[:,queries.columns !='income_over_50k']
query_label = queries.loc[:,queries.columns =='income_over_50k']

In [49]:
total_train_data.shape

(23443, 39)

## Build Run MultiLayer Perceptron
To have easy access to the weight vectors associated with every layer, we will use pytorch.
Our perceptron will have an input layer, a dense layer, and ELU layer, another dense layer, and a loss layer.

In [309]:
import torch
import torch.nn as nn
import math

axes_max = total_train_data.values.max(axis=0)

X = torch.from_numpy(total_train_data.values/axes_max).float()
y = torch.from_numpy(total_train_label.values).float()

In [369]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 39
        self.outputSize = 1
        self.hiddenSize = 200
        
        # weights
        
        mu1, sigma1 = 0, math.sqrt(2.0/(self.inputSize+self.hiddenSize))
        mu2, sigma2 = 0,math.sqrt(2.0/(self.outputSize+self.hiddenSize))
        
        self.W1 = torch.from_numpy(np.random.normal(mu1,sigma1,[self.inputSize,self.hiddenSize])).float()
        self.W2 = torch.from_numpy(np.random.normal(mu2,sigma2,[self.hiddenSize,self.outputSize])).float()
        
    def forward(self, X):
        alpha = 0.9
        self.z1 = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.ELU(self.z1,alpha) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def ELU(self,s,alpha):
        temp_out = s.clone()
        
        temp_out[temp_out<=0] = alpha*(np.exp(temp_out[temp_out<=0]-1))
        
        return temp_out
    
    def ELUPrime(self,s,alpha):

        temp_out2 = torch.from_numpy(np.ones(s.shape)).float()
        temp_out2[s<=0] = alpha * torch.exp(s[s<=0])

        return temp_out2
        
    def backward(self, X, y, o):
        lr_rate = 0.01
        alpha = 0.9
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.ELUPrime(self.z2,alpha)
        self.W1 += lr_rate * torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += lr_rate * torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self,X_query, mute):
        
        if mute is False:
            print ("Predicted data based on trained weights: ")
            print ("Input: \n" + str(X_query))
            print ("Output: \n" + str(round(self.forward(X_query).numpy()[0])))
            print ("Confidence: \n" + str(self.forward(X_query).numpy()[0]))
        
        prediction = np.around(self.forward(X_query).numpy())
        conf = self.forward(X_query).numpy()

        return prediction, conf
        

Build instance of model we just built.

In [370]:
j = 3
NN = Neural_Network()
for i in range(1):  # trains the NN 1,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
print("true: \n",y[j])
NN.predict(X[j,:],False)

#0 Loss: 0.27176985144615173
true: 
 tensor([1.])
Predicted data based on trained weights: 
Input: 
tensor([0.5333, 0.5625, 0.0000, 0.0000, 0.4040, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 1.0000])
Output: 
0.0
Confidence: 
0.0


(array([0.], dtype=float32), array([0.], dtype=float32))

Test NN model on validation data.

In [371]:
from sklearn.metrics import accuracy_score

X_test = torch.from_numpy(total_test_data.values/axes_max).float()
y_test = torch.from_numpy(total_test_label.values).float()
preds = np.zeros(X_test.shape[0])
confs = np.zeros(X_test.shape[0])
mute = True;


preds, confs = NN.predict(X_test,mute)
    
acc = accuracy_score(total_test_label,preds)
acc

0.7617625730495243

In [372]:
np.sum(confs)

0.0

In [373]:
preds.size

23443